Topic classification with BERTopic

Install transformers 4.41.0 for compatability with spacy and BERTopic

`pip install transformers==4.41.0` 

Careful when running this, very hardware intensive

In [7]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import psutil
import gc
from tqdm import tqdm
import re
import umap

In [8]:
df_posts = pd.read_json('../data/dataset.json')
df_posts.head()

,timestamp,text,text_id,user,user_id
0,2024-10-31,Running a business means juggling countless ad...,2018569761,danielwoodard,1077866112
1,2024-10-31,Liz Truss is walking in the lingering shadow o...,2092717718,nelsonjacqueline,1089670430
2,2024-10-31,The UK is bracing for war as government buildi...,2059143248,ihooper,1007478642
3,2024-10-31,Marrying a second or third cousin once removed...,2008209828,wrightnicholas,1039258480
4,2024-10-31,It's truly disgraceful how the Indian National...,2001239278,michael51,1021455936


Apply minimal preprocessing (Remove URLs and Mentions)

In [9]:
df_posts['text'] = df_posts['text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+|@\w+', '', x) if pd.notna(x) else x)


In [10]:
#df_posts = df_posts.sample(n=10000, random_state=42)  # Uncomment to test with sample

In [11]:
def monitor_memory():
    """Monitor memory usage"""
    process = psutil.Process()
    memory_gb = process.memory_info().rss / 1024 / 1024 / 1024
    return f"Memory Usage: {memory_gb:.2f} GB"

def create_multifeature_embeddings(df_posts, sentence_model, batch_size=64):
    """
    Create embeddings with memory monitoring and larger batches
    """
    print(f"\nStarting embedding generation for {len(df_posts)} documents")
    print(monitor_memory())
    
    print("\nGenerating text embeddings...")
    text_embeddings = sentence_model.encode(
        df_posts['text'].fillna("").tolist(),
        batch_size=batch_size,
        show_progress_bar=True
    )
    print(monitor_memory())
    
    print(monitor_memory())
    return text_embeddings

def setup_bertopic_model(df_posts, batch_size=64):
    """
    Set up and train BERTopic model with memory optimization
    """
    print(f"\nDataset size: {len(df_posts)} documents")
    print(f"DataFrame memory usage: {df_posts.memory_usage().sum() / 1024**2:.2f} MB")
    print(monitor_memory())
    
    print("\nInitializing models...")
    sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
    
    vectorizer_model = CountVectorizer(
        stop_words="english",
        #min_df = 350,  # Terms must appear in at least 350 rows (0.5% of dataset)
        #max_df = 0.8,  # Terms must appear in less than 80% of the dataset (56,000 rows)
        ngram_range=(1, 2)
    )
    
    umap_model = UMAP(
        n_neighbors=15,
        n_components=5,
        min_dist=0.0,
        metric='cosine',
        low_memory=True,
        random_state=42
    )
    
    embeddings = create_multifeature_embeddings(df_posts, sentence_model, batch_size)
    
    topic_model = BERTopic(
        embedding_model=sentence_model,
        vectorizer_model=vectorizer_model,
        umap_model=umap_model,
        min_topic_size=34,
        nr_topics='auto',
        calculate_probabilities=True,
        verbose=True
    )
    
    print("\nFitting BERTopic model...")
    topics, probs = topic_model.fit_transform(
        documents=df_posts['text'].fillna("").tolist(),
        embeddings=embeddings
    )
    
    return topic_model, topics, probs

def analyze_topics(topic_model, topics, df_posts):
    """
    Analyze topics with memory considerations
    """
    print("\nAnalyzing topics...")
    print(monitor_memory())
    
    topic_info = topic_model.get_topic_info()
    
    df_posts['topic'] = topics
    
    topic_docs = {}
    unique_topics = set(topics)
    print(f"\nFound {len(unique_topics)-1} topics (excluding -1)")
    
    for topic in tqdm(unique_topics):
        if topic != -1:
            topic_docs[topic] = df_posts[df_posts['topic'] == topic]['text'].head(3).tolist()
    
    try:
        print("\nGenerating visualizations...")
        topic_model.visualize_topics()
        topic_model.visualize_hierarchy()
    except Exception as e:
        print(f"Warning: Visualization error: {e}")
    
    return topic_info, topic_docs

def run_topic_analysis(df_posts, batch_size=64):
    """
    Run the complete pipeline with memory monitoring
    """
    print(f"Starting analysis with batch size: {batch_size}")
    print(monitor_memory())
    
    if 'text' not in df_posts.columns:
        raise ValueError("Missing required column: text")
    
    topic_model, topics, probs = setup_bertopic_model(df_posts, batch_size)
    topic_info, topic_docs = analyze_topics(topic_model, topics, df_posts)
    
    summary = {
        'num_topics': len(set(topics)) - 1,
        'topic_sizes': topic_info['Count'].tolist(),
        'top_topics': topic_info.head(10).to_dict('records')
    }
    
    return topic_model, summary, topics, probs

In [12]:
try:
    topic_model, summary, topics, probs = run_topic_analysis(df_posts, batch_size=64)
    
    print(f"\nAnalysis complete!")
    print(f"Found {summary['num_topics']} topics")
    print("\nTop 10 topics:")
    for topic in summary['top_topics']:
        print(f"Topic {topic['Topic']}: Size {topic['Count']}")
    
    topic_model.save("bertopic_model_large")
    
except Exception as e:
    print(f"Error during analysis: {e}")

Starting analysis with batch size: 64
Memory Usage: 3.09 GB

Dataset size: 70260 documents
DataFrame memory usage: 2.68 MB
Memory Usage: 3.09 GB

Initializing models...

Starting embedding generation for 70260 documents
Memory Usage: 3.11 GB

Generating text embeddings...


Batches: 100%|██████████| 1098/1098 [01:59<00:00,  9.21it/s]
2025-01-25 16:34:21,631 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


Memory Usage: 3.21 GB
Memory Usage: 3.21 GB

Fitting BERTopic model...


2025-01-25 16:35:05,225 - BERTopic - Dimensionality - Completed ✓
2025-01-25 16:35:05,226 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid usi


Analyzing topics...
Memory Usage: 2.85 GB

Found 25 topics (excluding -1)


100%|██████████| 26/26 [00:00<00:00, 4483.67it/s]


Generating visualizations...



2025-01-25 16:38:00,605 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.



Analysis complete!
Found 25 topics

Top 10 topics:
Topic -1: Size 27899
Topic 0: Size 38795
Topic 1: Size 1077
Topic 2: Size 776
Topic 3: Size 219
Topic 4: Size 158
Topic 5: Size 123
Topic 6: Size 111
Topic 7: Size 98
Topic 8: Size 90


In [13]:
# 1. Save visualizations to HTML files
fig = topic_model.visualize_barchart(top_n_topics=20)
fig.write_html("../output/topic_barchart.html")

topic_model.visualize_topics().write_html("../output/topic_clusters.html")
topic_model.visualize_hierarchy().write_html("../output/topic_hierarchy.html")

# 2. Print text-based summary
topics_info = topic_model.get_topic_info()
print("\nMost frequent topics with their terms:")
for _, row in topics_info.head(20).iterrows():
    topic_id = row['Topic']
    size = row['Count']
    if topic_id != -1:
        terms = topic_model.get_topic(topic_id)
        print(f"\nTopic {topic_id} (Size: {size}):")
        # Print top 10 terms for each topic with their weights
        for term, weight in terms[:20]:
            print(f"  - {term}: {weight:.3f}")


Most frequent topics with their terms:

Topic 0 (Size: 38795):
  - health: 0.012
  - business: 0.012
  - let: 0.011
  - just: 0.008
  - politics: 0.008
  - time: 0.008
  - vote: 0.008
  - check: 0.008
  - science: 0.008
  - support: 0.007

Topic 1 (Size: 1077):
  - oct 31: 0.170
  - climate high: 0.170
  - precip: 0.170
  - 31 climate: 0.170
  - snow: 0.170
  - 31: 0.169
  - oct: 0.168
  - low: 0.159
  - high: 0.154
  - climate: 0.133

Topic 2 (Size: 776):
  - unknown: 0.136
  - times climateemergency: 0.098
  - climateemergency scotradar: 0.098
  - scotradar: 0.098
  - scotradar cop26: 0.098
  - alt: 0.098
  - cop26 cop26glasgow: 0.096
  - cop26: 0.095
  - cop26glasgow: 0.094
  - seen: 0.094

Topic 3 (Size: 219):
  - airport: 0.194
  - 31 climate: 0.149
  - precip: 0.149
  - climate high: 0.149
  - snow: 0.149
  - oct 31: 0.149
  - 31: 0.148
  - oct: 0.147
  - low: 0.139
  - high: 0.134

Topic 4 (Size: 158):
  - blue: 0.120
  - english: 0.110
  - math: 0.108
  - green: 0.084
  - scie

: 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import umap

doc_embeddings = topic_model.c_tf_idf_[topic_model.topics_]    # Get embeddings for each topic

reducer = umap.UMAP(n_components=2, random_state=42) # Reduce to 2D
topic_embeddings_2d = reducer.fit_transform(doc_embeddings)

In [ ]:
# Fit k-Means clustering
kmeans = KMeans(n_clusters=5, random_state=42)  # Adjust n_clusters as needed
kmeans.fit(topic_embeddings_2d)

In [ ]:
# Calculate silhouette score
score = silhouette_score(topic_embeddings_2d, kmeans.labels_)
print(f"Silhouette Score for 2 dimensions: {score}")

In [ ]:
# Add reduced dimensions to the DataFrame
df_posts['topic_value_1'] = topic_embeddings_2d[:, 0]
df_posts['topic_value_2'] = topic_embeddings_2d[:, 1]


In [ ]:
import matplotlib.pyplot as plt

plt.scatter(topic_embeddings_2d[:, 0], topic_embeddings_2d[:, 1], c=kmeans.labels_, cmap='viridis', s=10)
plt.title('k-Means Clustering of Topic Embeddings')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.show()

In [16]:
# Add topic numbers and labels to dataframe 
df_posts['topic'] = topics

# Get topic labels
topic_labels = {}
for topic_id in set(topics):
    if topic_id != -1:
        terms = topic_model.get_topic(topic_id)
        topic_labels[topic_id] = ", ".join([term for term, _ in terms[:3]])
    else:
        topic_labels[topic_id] = "Other"

# Add descriptive labels
df_posts['topic_label'] = df_posts['topic'].map(topic_labels)

# Save augmented dataframe
df_posts.to_csv('../output/posts_with_topics.csv', index=False)